In [1]:
import chromadb

In [3]:
client = chromadb.PersistentClient(path="database/myDB")
collection = client.get_or_create_collection(name="documents")
registry = client.get_or_create_collection("corpus_model_registry")

In [5]:
collection.delete(where={"corpus_name": "Corpus 3"})

In [6]:
import os

In [7]:
model_dir = os.path.join("models")
    
if not os.path.exists(model_dir):
    print("nothing") # Return empty if no directory

# List all directories/files inside model_dir
models = [
    name for name in os.listdir(model_dir)
    if os.path.isdir(os.path.join(model_dir, name))
]
print(models)


['model1']
